In [1]:
# 📦 Install Required Libraries (only run once)
!pip install joblib \
             scikit-learn==1.5.2 \
             geopy \
             xgboost==2.1.1

Defaulting to user installation because normal site-packages is not writeable

In [2]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from models import load_models
from utils import get_lat_long, cal_lease_remaining_years, get_address
from config import town_coordinates, towns, flat_types, flat_models, expected_columns

In [3]:
rf, gb, xgb, dt, meta_model, scaler = load_models()

## Input Flat Details

In [9]:
# Add input feature values accordingly
selected_town = "JURONG East"
selected_flat_type = "4 ROOM"
selected_flat_model = "Improved"
block = "909"
street_name = "JURONG East ST 91"
year_of_sale = 2019
storey_median = 8
lease_commence_date = 1989
floor_area_sqm = 122.0


## Calculate Distances to Amenities

In [10]:
# Construct address and get coordinates
address = get_address(block, street_name)
latitude, longitude = get_lat_long(address)
origin = (latitude, longitude)

# Load amenity locations
mrt_location = pd.read_csv('data/mrt_address.csv')
school_location = pd.read_csv('data/schools_address.csv')
supermarket_location = pd.read_csv('data/shops_address.csv')
hawker_location = pd.read_csv('data/hawkers_address.csv')

# Calculate distances
nearest_mrt_distance = min([geodesic(origin, (row['latitude'], row['longitude'])).meters for _, row in mrt_location.iterrows()])
nearest_supermarket_distance = min([geodesic(origin, (row['latitude'], row['longitude'])).meters for _, row in supermarket_location.iterrows()])
nearest_school_distance = min([geodesic(origin, (row['latitude'], row['longitude'])).meters for _, row in school_location.iterrows()])
nearest_hawkers_distance = min([geodesic(origin, (row['latitude'], row['longitude'])).meters for _, row in hawker_location.iterrows()])
cbd_distance = geodesic(origin, (1.287953, 103.851784)).meters
calculated_remaining_lease = cal_lease_remaining_years(lease_commence_date, year_of_sale)


## Prepare Input Data

In [11]:
input_data = {
    'nearest_hawkers_distance': [nearest_hawkers_distance],
    'floor_area_sqm': [floor_area_sqm],
    'nearest_mrt_distance': [nearest_mrt_distance],
    'nearest_school_distance': [nearest_school_distance],
    'cbd_distance': [cbd_distance],
    'year_of_sale': [year_of_sale],
    'storey_median': [storey_median],
    'calculated_remaining_lease': [calculated_remaining_lease],
    'nearest_supermarket_distance': [nearest_supermarket_distance]
}

input_df = pd.DataFrame(input_data)

for town in towns:
    input_df[f'town_{town}'] = 1 if selected_town == town else 0

for flat_type in flat_types:
    input_df[f'flat_type_{flat_type}'] = 1 if selected_flat_type == flat_type else 0

for flat_model in flat_models:
    input_df[f'flat_model_{flat_model}'] = 1 if selected_flat_model == flat_model else 0

input_df = input_df.reindex(columns=expected_columns, fill_value=0)
input_scaled = scaler.transform(input_df)
input_df.head()


,floor_area_sqm,nearest_supermarket_distance,nearest_school_distance,nearest_mrt_distance,nearest_hawkers_distance,cbd_distance,year_of_sale,calculated_remaining_lease,storey_median,town_BEDOK,...,flat_model_Multi Generation,flat_model_New Generation,flat_model_Premium Apartment,flat_model_Premium Apartment Loft,flat_model_Premium Maisonette,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2
0,122.0,567.593236,211.32884,1273.061437,3096.32379,19318.073923,2019,69,8,0,...,0,0,0,0,0,0,0,0,0,0


## Predict Resale Price

In [12]:
rf_pred_new = rf.predict(input_scaled)
gb_pred_new = gb.predict(input_scaled)
xgb_pred_new = xgb.predict(input_scaled)
dt_pred_new = dt.predict(input_scaled)

X_new_meta = np.column_stack((rf_pred_new, gb_pred_new, dt_pred_new, xgb_pred_new))
y_new_pred = meta_model.predict(X_new_meta)

# Print results
print("Predicted Resale Price: ${:.2f}".format(y_new_pred[0]))
print("Distance to Nearest MRT: {:.2f} m".format(nearest_mrt_distance))
print("Nearest Supermarket: {:.2f} m".format(nearest_supermarket_distance))
print("Nearest School: {:.2f} m".format(nearest_school_distance))
print("Nearest Hawker Centre: {:.2f} m".format(nearest_hawkers_distance))
print("Distance to CBD: {:.2f} m".format(cbd_distance))

Predicted Resale Price: $320191.09
Distance to Nearest MRT: 1273.06 m
Nearest Supermarket: 567.59 m
Nearest School: 211.33 m
Nearest Hawker Centre: 3096.32 m
Distance to CBD: 19318.07 m
